# Gold の処理

## 共通設定の読み取りとライブラリのインポート

In [0]:
%run ./00_config

In [0]:
from pyspark.sql import functions as F

## Gold (Curated Files) の処理

`tips/02_how_to_convert_pptx` ノートブックに記述しているように PDF 変換や画像の分割を実施します。一部の処理を Databricks Free Edition で実行できないためスキップします。

In [0]:
# 省略

In [0]:
# 画像を配置した URL をセット
image_src_volume_dir = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}"
image_src_volume_dir += "/source_data/curated/curated_files"
image_src_volume_dir += (
    "/746dfa1510e04c260652d56807aded43280608ed4d318351c7e9562ea4184f80"
)
image_src_volume_dir += "/png"

In [0]:
display(dbutils.fs.ls(image_src_volume_dir))

## Gold (Curated) の処理

In [0]:
enriched_table_df = spark.table(f"{catalog_name}.{schema_name}.{enriched_table_name}")
enriched_table_df.display()

In [0]:
src_cols = ["file_id", "file_url", "file_md_content"]

In [0]:
src_curated_table_df_01 = enriched_table_df.where("audit__source_delete_flg = 0")
src_curated_table_df_01 = src_curated_table_df_01.select(src_cols)

In [0]:
src_curated_table_df_02 = src_curated_table_df_01.withColumn(
    "contents", F.explode("file_md_content")
)
file_contents_col_cond = {
    "slide_number": F.expr("contents.slide_number"),
    "slide_content": F.expr("contents.slide_content"),
}
src_curated_table_df_02 = src_curated_table_df_02.withColumns(file_contents_col_cond)
src_curated_table_df_02 = src_curated_table_df_02.dropDuplicates(
    ["file_id", "slide_number"]
)

# 指定したページの PDF を表示する URL に変換
page_param_key = "#p"  ## Docswell では #p と指定するが、 SharePoint では #page= を指定
url_col_conf = {
    "file_url": F.concat(
        F.col("file_url"), F.lit(page_param_key), F.col("slide_number")
    )
}
src_curated_table_df_02 = src_curated_table_df_02.withColumns(url_col_conf)

# ページごとの Volume の URL を追加
img_path_prefix = "/slide_"
img_path_suffix = ".png"
img_path_col_conf = {
    "image_path": F.concat(
        F.lit(image_src_volume_dir),
        F.lit(img_path_prefix),
        F.col("slide_number"),
        F.lit(img_path_suffix),
    )
}
src_curated_table_df_02 = src_curated_table_df_02.withColumns(img_path_col_conf)


# Databricks の Mosaic AI Vector Search 利用時の単一の PK 値を設定
src_curated_table_df_02 = src_curated_table_df_02.withColumn(
    "pk",
    F.sha2(
        F.concat_ws(
            "||",
            F.coalesce(F.col("file_id").cast("string"), F.lit("")),
            F.coalesce(F.col("slide_number").cast("string"), F.lit("")),
        ),
        256,
    ),
)

# 表示する上で不要なカラムを削除
src_curated_table_df_02 = src_curated_table_df_02.drop(
    "file_md_content",
    "contents",
)

src_curated_table_df_02.display()

In [0]:
# Embedding の実行

# <暫定対応>日本語の Embedding モデルを指定すべきであるが英語のものを指定
embedding_model = "databricks-bge-large-en"

# ＜暫定対応＞Databricks Free Edition にてレコード数が多い場合に処理が完了しないため、分割して実行する
src_df_01, src_df_02, src_df_03 = src_curated_table_df_02.randomSplit(
    [0.4, 0.3, 0.3], seed=42
)
src_dfs = [src_df_01, src_df_02, src_df_03]
for src_d in src_dfs:
    src_curated_table_df = src_d.withColumn(
        "embedding", F.expr(f"ai_query('{embedding_model}', slide_content)")
    )
    src_curated_table_df.write.mode("append").saveAsTable(
        f"{catalog_name}.{schema_name}.{curated_table_name}"
    )

In [0]:
curated_table_df = spark.table(f"{catalog_name}.{schema_name}.{curated_table_name}")
curated_table_df.display()

In [0]:
# end